In [ ]:
# For tips on running notebooks in Google Colab, see
# https://docs.pytorch.org/tutorials/beginner/colab
%matplotlib inline

Writing Custom Datasets, DataLoaders and Transforms
===================================================

**Author**: [Sasank Chilamkurthy](https://chsasank.github.io)

A lot of effort in solving any machine learning problem goes into
preparing the data. PyTorch provides many tools to make data loading
easy and hopefully, to make your code more readable. In this tutorial,
we will see how to load and preprocess/augment data from a non trivial
dataset.

To run this tutorial, please make sure the following packages are
installed:

-   `scikit-image`: For image io and transforms
-   `pandas`: For easier csv parsing


In [ ]:
# Ignore warnings

The dataset we are going to deal with is that of facial pose. This means
that a face is annotated like this:

![](https://pytorch.org/tutorials/_static/img/landmarked_face2.png){width="400px"}

Over all, 68 different landmark points are annotated for each face.

```{=html}
<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
```
```{=html}
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
```
```{=html}
<p>Download the dataset from <a href="https://download.pytorch.org/tutorial/faces.zip">here</a>so that the images are in a directory named 'data/faces/'.This dataset was actuallygenerated by applying excellent <a href="https://blog.dlib.net/2014/08/real-time-face-pose-estimation.html">dlib's poseestimation</a>on a few images from imagenet tagged as 'face'.</p>
```
```{=html}
</div>
```
Dataset comes with a `.csv` file with annotations which looks like this:

``` {.sh}
image_name,part_0_x,part_0_y,part_1_x,part_1_y,part_2_x, ... ,part_67_x,part_67_y
0805personali01.jpg,27,83,27,98, ... 84,134
1084239450_e76e00b7e7.jpg,70,236,71,257, ... ,128,312
```

Let\'s take a single image name and its annotations from the CSV, in
this case row index number 65 for person-7.jpg just as an example. Read
it, store the image name in `img_name` and store its annotations in an
(L, 2) array `landmarks` where L is the number of landmarks in that row.


Let\'s write a simple helper function to show an image and its landmarks
and use it to show a sample.


Dataset class
=============

`torch.utils.data.Dataset` is an abstract class representing a dataset.
Your custom dataset should inherit `Dataset` and override the following
methods:

-   `__len__` so that `len(dataset)` returns the size of the dataset.
-   `__getitem__` to support the indexing such that `dataset[i]` can be
    used to get $i$th sample.

Let\'s create a dataset class for our face landmarks dataset. We will
read the csv in `__init__` but leave the reading of images to
`__getitem__`. This is memory efficient because all the images are not
stored in the memory at once but read as required.

Sample of our dataset will be a dict
`{'image': image, 'landmarks': landmarks}`. Our dataset will take an
optional argument `transform` so that any required processing can be
applied on the sample. We will see the usefulness of `transform` in the
next section.


Let\'s instantiate this class and iterate through the data samples. We
will print the sizes of first 4 samples and show their landmarks.


Transforms
==========

One issue we can see from the above is that the samples are not of the
same size. Most neural networks expect the images of a fixed size.
Therefore, we will need to write some preprocessing code. Let\'s create
three transforms:

-   `Rescale`: to scale the image
-   `RandomCrop`: to crop from image randomly. This is data
    augmentation.
-   `ToTensor`: to convert the numpy images to torch images (we need to
    swap axes).

We will write them as callable classes instead of simple functions so
that parameters of the transform need not be passed every time it\'s
called. For this, we just need to implement `__call__` method and if
required, `__init__` method. We can then use a transform like this:

``` {.python}
tsfm = Transform(params)
transformed_sample = tsfm(sample)
```

Observe below how these transforms had to be applied both on the image
and landmarks.


```{=html}
<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
```
```{=html}
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
```
```{=html}
<p>In the example above,  uses an external library's random number generator(in this case, Numpy's ). This can result in unexpected behavior with (see <a href="https://pytorch.org/docs/stable/notes/faq.html#my-data-loader-workers-return-identical-random-numbers">here</a>).In practice, it is safer to stick to PyTorch's random number generator, e.g. by using  instead.</p>
```
```{=html}
</div>
```


Compose transforms
==================

Now, we apply the transforms on a sample.

Let\'s say we want to rescale the shorter side of the image to 256 and
then randomly crop a square of size 224 from it. i.e, we want to compose
`Rescale` and `RandomCrop` transforms. `torchvision.transforms.Compose`
is a simple callable class which allows us to do this.


In [ ]:
# Apply each of the above transforms on sample.

Iterating through the dataset
=============================

Let\'s put this all together to create a dataset with composed
transforms. To summarize, every time this dataset is sampled:

-   An image is read from the file on the fly
-   Transforms are applied on the read image
-   Since one of the transforms is random, data is augmented on sampling

We can iterate over the created dataset with a `for i in range` loop as
before.


However, we are losing a lot of features by using a simple `for` loop to
iterate over the data. In particular, we are missing out on:

-   Batching the data
-   Shuffling the data
-   Load the data in parallel using `multiprocessing` workers.

`torch.utils.data.DataLoader` is an iterator which provides all these
features. Parameters used below should be clear. One parameter of
interest is `collate_fn`. You can specify how exactly the samples need
to be batched using `collate_fn`. However, default collate should work
fine for most use cases.


In [ ]:
# Helper function to show a batch


















# if you are using Windows, uncomment the next line and indent the for loop.
# you might need to go back and change ``num_workers`` to 0.

# if __name__ == '__main__':

Afterword: torchvision
======================

In this tutorial, we have seen how to write and use datasets, transforms
and dataloader. `torchvision` package provides some common datasets and
transforms. You might not even have to write custom classes. One of the
more generic datasets available in torchvision is `ImageFolder`. It
assumes that images are organized in the following way:

``` {.sh}
root/ants/xxx.png
root/ants/xxy.jpeg
root/ants/xxz.png
.
.
.
root/bees/123.jpg
root/bees/nsdf3.png
root/bees/asd932_.png
```

where \'ants\', \'bees\' etc. are class labels. Similarly generic
transforms which operate on `PIL.Image` like `RandomHorizontalFlip`,
`Scale`, are also available. You can use these to write a dataloader
like this:

``` {.pytorch}
import torch
from torchvision import transforms, datasets

data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
hymenoptera_dataset = datasets.ImageFolder(root='hymenoptera_data/train',
                                           transform=data_transform)
dataset_loader = torch.utils.data.DataLoader(hymenoptera_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=4)
```

For an example with training code, please see
`transfer_learning_tutorial`{.interpreted-text role="doc"}.


In [ ]:
# %%%%%%RUNNABLE_CODE_REMOVED%%%%%%